In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
import itertools
import seaborn as sn
from textblob import TextBlob
import glob

# warning
import warnings
warnings.filterwarnings("ignore")

In [32]:
df = pd.read_csv('../data/final_data.csv')
df.head(3)

,date,full_text,geo,truncated,display_text_range,user,tweet_id,entities,possibly_sensitive,longitude,...,yoursel,youth,youtube,yrxltdq,yummy,yumyum,yup,zero,zoom,sentiment
0,2020-03-30,Definitely making a routine to stop from going...,"[41.42362498, -122.39264801]",False,"[0, 239]","{'id': 988896150571790336, 'id_str': '98889615...",1244710975422574592,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,-122.392648,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0625
1,2020-03-30,This is my 'self-isolating' face. #quarantine ...,"[34.1722, -118.378]",False,"[0, 140]","{'id': 1008423932267921408, 'id_str': '1008423...",1244710063589089281,"{'hashtags': [{'text': 'quarantine', 'indices'...",False,-118.378000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
2,2020-03-30,Corona virus testing and treatment is availabl...,"[33.97977, -118.18884]",False,"[0, 233]","{'id': 107183997, 'id_str': '107183997', 'name...",1244708953994469377,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,-118.188840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4000


In [33]:
irrelevant_cols = ['full_text', 'geo', 'display_text_range', 'user', 'tweet_id',
       'entities', 'longitude', 'latitude', 'state', 'preprocess_full_text']

In [34]:
relevant_cols = df.drop(columns=irrelevant_cols)
relevant_cols.head(2)

,date,truncated,possibly_sensitive,county,retail_rec,groc_pharm,parks,transit,workplaces,residential,...,yoursel,youth,youtube,yrxltdq,yummy,yumyum,yup,zero,zoom,sentiment
0,2020-03-30,False,False,Siskiyou County,-33.0,-12.0,-2.0,-12.0,-40.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0625
1,2020-03-30,False,False,Los Angeles County,-42.0,-23.0,-34.0,-52.0,-51.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000


In [35]:
# Length before dropping missing values
len(relevant_cols)

17531

In [36]:
# Drop rows with NaN values
no_na = relevant_cols.dropna()

In [37]:
# Length after dropping missing values
len(no_na)

15923

In [38]:
# One hot encode categorical columns
dummy_cols = ['county', 'truncated', 'possibly_sensitive']
ohe_df = pd.get_dummies(no_na, drop_first=True, columns=dummy_cols)
ohe_df.head(2)

,date,retail_rec,groc_pharm,parks,transit,workplaces,residential,public,polarity,aad,...,county_Shasta County,county_Siskiyou County,county_Solano County,county_Sonoma County,county_Stanislaus County,county_Tulare County,county_Ventura County,county_Yolo County,truncated_True,possibly_sensitive_True
0,2020-03-30,-33.0,-12.0,-2.0,-12.0,-40.0,10.0,-24.25,-0.0625,0.0,...,0,1,0,0,0,0,0,0,0,0
1,2020-03-30,-42.0,-23.0,-34.0,-52.0,-51.0,22.0,-42.00,0.0000,0.0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
# Add county back
ohe_df['county'] = no_na['county']
ohe_df.head(1)

,date,retail_rec,groc_pharm,parks,transit,workplaces,residential,public,polarity,aad,...,county_Siskiyou County,county_Solano County,county_Sonoma County,county_Stanislaus County,county_Tulare County,county_Ventura County,county_Yolo County,truncated_True,possibly_sensitive_True,county
0,2020-03-30,-33.0,-12.0,-2.0,-12.0,-40.0,10.0,-24.25,-0.0625,0.0,...,1,0,0,0,0,0,0,0,0,Siskiyou County


In [43]:
y_labels = ohe_df.loc[:, ['retail_rec',
       'groc_pharm', 'parks', 'transit', 'workplaces', 'residential', 'public']]
y_labels.head()

,retail_rec,groc_pharm,parks,transit,workplaces,residential,public
0,-33.0,-12.0,-2.0,-12.0,-40.0,10.0,-24.25
1,-42.0,-23.0,-34.0,-52.0,-51.0,22.0,-42.00
2,-42.0,-23.0,-34.0,-52.0,-51.0,22.0,-42.00
4,-42.0,-23.0,-34.0,-52.0,-51.0,22.0,-42.00
5,-42.0,-23.0,-34.0,-52.0,-51.0,22.0,-42.00


In [44]:
# minus_first_20 = df.iloc[:, 20:]
# minus_first_20['date'] = df['date']
# minus_first_20['county'] = df['county']
# new_df = minus_first_20
# new_df.head()

In [45]:
grouped = ohe_df.groupby(['county','date']).mean()
grouped.head(2)

retail_rec  groc_pharm  parks  transit  workplaces  \
county         date                                                             
Alameda County 2020-03-30       -48.0       -23.0   -1.0    -71.0       -64.0   
               2020-04-01       -48.0       -18.0    0.0    -70.0       -65.0   

                           residential  public  polarity  aad  abc  ...  \
county         date                                                 ...   
Alameda County 2020-03-30         27.0  -51.50  0.233333  0.0  0.0  ...   
               2020-04-01         28.0  -50.25  0.221645  0.0  0.0  ...   

                           county_Shasta County  county_Siskiyou County  \
county         date                                                       
Alameda County 2020-03-30                   0.0                     0.0   
               2020-04-01                   0.0                     0.0   

                           county_Solano County  county_Sonoma County  \
county         date                                                     
Alameda County 2020-03-30                   0.0                   0.0   
               2020-04-01                   0.0                   0.0   

                           county_Stanislaus County  county_Tulare County  \
county         date                                                         
Alameda County 2020-03-30                       0.0                   0.0   
               2020-04-01                       0.0                   0.0   

                           county_Ventura County  county_Yolo County  \
county         date                                                    
Alameda County 2020-03-30                    0.0                 0.0   
               2020-04-01                    0.0                 0.0   

                           truncated_True  possibly_sensitive_True  
county         date                                                 
Alameda County 2020-03-30             0.0                      0.0  
               2020-04-01             0.0                      1.0  

[2 rows x 2332 columns]

In [46]:
len(grouped)

2540

In [49]:
reset_ind = grouped.reset_index()
reset_ind.drop(columns=['county'], inplace=True)


In [50]:
reset_ind.head(2)

,date,retail_rec,groc_pharm,parks,transit,workplaces,residential,public,polarity,aad,...,county_Shasta County,county_Siskiyou County,county_Solano County,county_Sonoma County,county_Stanislaus County,county_Tulare County,county_Ventura County,county_Yolo County,truncated_True,possibly_sensitive_True
0,2020-03-30,-48.0,-23.0,-1.0,-71.0,-64.0,27.0,-51.50,0.233333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-04-01,-48.0,-18.0,0.0,-70.0,-65.0,28.0,-50.25,0.221645,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
